### **LLM API 활용해보기**

In [ ]:
#필수 라이브러리 설치
!pip install langchain openai anthropic langchain-openai langchain-anthropic

**[앤트로픽의 Claude 2.1 모델 API 호출 코드]**

In [ ]:
import anthropic

anthropic.Anthropic(
    api_key="YOUR_API_KEY").messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, world"}
    ]
)

**[오픈AI의 GPT-3.5 Turbo 모델 API 호출 코드]**

In [ ]:
from openai import OpenAI

client = OpenAI(api_key = "YOUR_API_KEY")
client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Who won the world series in 2020?"
        }
    ]
)

**[랭체인을 활용한 앤트로픽 Claude 2.1 모델 API 호출 코드]**

In [ ]:
from langchain_anthropic import ChatAnthropic
chat = ChatAnthropic(
    model_name ="claude-3-opus-20240229",
    anthropic_api_key="YOUR_API_KEY"
)
chat.invoke("안녕~ 너를 소개해줄래?")

**[랭체인을 활용한 오픈AI GPT-3.5 Turbo 모델 API 호출 코드]**

In [ ]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model_name = 'gpt-4o',
    openai_api_key="YOUR_API_KEY"
)
chat.invoke("안녕~ 너를 소개해줄래?")

### **프롬프트 템플릿에 대해 알아보기**

**[ChatPromptTemplate]**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
	#SystemMessage: 유용한 챗봇이라는 역할과 이름을 부여
        ("system", "You are a helpful AI bot. Your name is {name}."),
    #HumanMessage와 AIMessage: 서로 안부를 묻고 답하는 대화 히스토리 주입
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
    #HumanMessage로 사용자가 입력한 프롬프트를 전달
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(messages)

In [ ]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
       "You are a helpful assistant that re-writes the user's text to sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

### **LLM API의 다양한 기능 활용해보기**

**[LLM API의 Temperature 이애하기]**

In [ ]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

#OPENAI API키 저장
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

#Temperature=0
chatgpt_temp0_1 = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature = 0)
chatgpt_temp0_2 = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature = 0)

#Temperature=1
chatgpt_temp1_1 = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature = 1)
chatgpt_temp1_2 = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature = 1)

model_list = [chatgpt_temp0_1, chatgpt_temp0_2, chatgpt_temp1_1, chatgpt_temp1_2]

for i in model_list:
    answer = i.invoke("왜 파이썬이 가장 인기있는 프로그래밍 언어인지 한 문장으로 설명해줘", max_tokens = 128)
    print("-"*100)
    print(">>>",answer.content)

**[ChatGPT처럼 답변 스트리밍하기]**

In [ ]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature = 0)
for chunk in chat.stream("달에 관한 시를 써줘"):
    print(chunk.content, end="", flush=True)

**[답변 캐싱하기]**

In [ ]:
from langchain.globals import set_llm_cache #캐시메모리 라이브러리 호출
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature = 0)

**첫 질문-응답 시간 측정**

In [ ]:
pip install langchain-community

In [ ]:
%%time
#셀 실행 시간 측정
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache()) #캐시메모리 설정

chat.invoke("일반상대성 이론을 한마디로 설명해줘")

**두번째 질문-응답 시간 측정**

In [ ]:
%%time
#같은 질문 전달
chat.invoke("일반상대성 이론을 한마디로 설명해줘")

## **PromptTemplate과 ChatPromptTemplate**

**[PromptTemplate]**

In [ ]:
from langchain.prompts import PromptTemplate

prompt= (
    PromptTemplate.from_template(
        """
        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고,
        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
        )
    )

prompt

In [ ]:
prompt.format(개수= 3, 재료="사과, 양파, 계란")

**[ChatPromptTemplate]**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

prompt = SystemMessage(content=
        """
        너는 항상 밝은 말투로 대화하는 챗봇이야. 답변의 끝에 이모티콘을 붙여줘.
        """
        )

new_prompt = (
    prompt
    + HumanMessage(content=
                          """
                          오늘은 날씨가 어때?
                          """)
    + AIMessage(content=
                         """
                         오늘은 날씨가 아주 좋아요!
                         """)
    + """{input}"""
)

new_prompt.format_messages(input = "오늘 너의 기분은 어때?")

**[간단한 LLM Chain 구성해보기]**

In [ ]:
import os
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model = 'gpt-4o')
chain = LLMChain(llm=model, prompt=new_prompt)
chain.invoke("오늘 너의 기분은 어때?")

## **Few-shot 예제를 통한 프롬프트 템플릿**

**[3행시 예제 프롬프트 템플릿]**

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
examples = [
    {
        "question": "아이유로 삼행시 만들어줘",
        "answer":
                """
                아: 아이유는
                이: 이런 강의를 들을 이
                유: 유가 없다.
                """
    }
]

example_prompt = PromptTemplate(
                                input_variables=["question", "answer"],
                                template="Question: {question}\n{answer}"
                                )

print(example_prompt.format(**examples[0]))

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="호날두로 삼행시 만들어줘"))

**[Few-shot 미적용 결과]**

In [ ]:
model = ChatOpenAI(model_name = "gpt-3.5-turbo-0125", temperature = 1)
result = model.invoke("호날두로 삼행시 만들어줘")
print(result.content)

**[Few-shot 적용 결과]**

In [ ]:
result = model.invoke(prompt.format(input="호날두로 삼행시 만들어줘"))
print(result.content)

## **Partial Prompt Template**

**[Partial Prompt 예시]**

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("나이: {age} \n직업: {job}")
partial_prompt = prompt.partial(age="20")
print(partial_prompt.format(job="개발자"))

**[현재 날짜 Partial Prompt 만들어보기]**

In [ ]:
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

In [ ]:
_get_datetime()

In [ ]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

## **PDF Loaders**

### **PyPDF Loader**

In [ ]:
#PyPDF 설치
!pip install -q pypdf

In [ ]:
#PyPDFLoader 불러오기
from langchain_community.document_loaders import PyPDFLoader

# PDF파일 불러올 객체 PyPDFLoader 선언
loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

# PDF파일 로드 및 페이지별로 자르기
pages = loader.load_and_split()
print(pages[5].page_content)

In [ ]:
pages

**[OCR 기능 활용하여 이미지-텍스트 혼합 페이지 내 텍스트 추출하기]**

In [ ]:
#OCR기능 위해 설치
!pip install rapidocr-onnxruntime

*아래 셀은 실행 시 많은 시간이 소요됩니다.

In [ ]:
#PyPDFLoader 불러오기
from langchain_community.document_loaders import PyPDFLoader

# PDF파일 불러올 객체 PyPDFLoader 선언(extract_images 매개변수로 OCR 수행)
loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf", extract_images=True)

# PDF파일 로드 및 페이지별로 자르기
pages = loader.load_and_split()
print(pages[5].page_content)

**[페이지 내 테이블 추출하기]**

In [ ]:
#PyPDFLoader 불러오기
from langchain_community.document_loaders import PyPDFLoader

# PDF파일 불러올 객체 PyPDFLoader 선언
loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

# PDF파일 로드 및 페이지별로 자르기
pages = loader.load_and_split()
print(pages[3])

**[이미지+텍스트 페이지 내 텍스트 추출]**

In [ ]:
#PyPDFium2 설치
pip install pypdfium2

In [ ]:
#PyPDFium2 불러오기
from langchain_community.document_loaders import PyPDFium2Loader

# PDF파일 불러올 객체 PyPDFium2Loader 선언
loader = PyPDFium2Loader(r"C:\Users\gram\Desktop\Langchain\DocumentLoader\[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

# PDF파일 로드 및 페이지별로 자르기(PyPDFium2는 load_and_split 함수 없이도 페이지별로 자름)
data = loader.load()
print(data[5].page_content)

**[페이지 내 테이블 추출하기]**

In [ ]:
#PyPDFium2 불러오기
from langchain_community.document_loaders import PyPDFium2Loader

# PDF파일 불러올 객체 PyPDFium2Loader 선언
loader = PyPDFium2Loader(r"C:\Users\gram\Desktop\Langchain\DocumentLoader\[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

# PDF파일 로드 및 페이지별로 자르기(PyPDFium2는 load_and_split 함수 없이도 페이지별로 자름)
data = loader.load()
print(data[3].page_content)

**[PyPDFLoader의 텍스트 추출 소요 시간]**

In [ ]:
%%time
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\gram\Desktop\Langchain\DocumentLoader\[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

pages = loader.load_and_split()

**[PyPDFium2의 텍스트 추출 소요 시간]**

In [ ]:
%%time
from langchain_community.document_loaders import PyPDFium2Loader

loader = PyPDFium2Loader(r"C:\Users\gram\Desktop\Langchain\DocumentLoader\[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

pages = loader.load()

## **Office file Loaders**

### **Word 파일 불러오기**

In [ ]:
#docx2txt 설치
!pip install --upgrade --quiet  docx2txt

In [ ]:
#Docx2txtLoader 불러오기
from langchain_community.document_loaders import Docx2txtLoader

#Docx2txtLoader로 워드 파일 불러오기(경로 설정)
loader = Docx2txtLoader(r"/content/drive/MyDrive/마소캠퍼스/[삼성전자] 사업보고서(일반법인) (2021.03.09).docx")

#페이지로 분할하여 불러오기
data = loader.load_and_split()
data[0]
#첫번째 페이지 출력하기
# print(data[12].page_content[:500])

In [ ]:
print(data[12].page_content[:500])
print(data[12].metadata)

### **CSV 파일 불러오기**

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path=r"/content/drive/MyDrive/마소캠퍼스/mlb_teams_2012.csv")

data = loader.load()

data[0]

### **PPT 파일 불러오기**

In [ ]:
#pthon-pptx 패키지 설치
!pip install -q python-pptx unstructured

In [ ]:
#UnstructuredPowerPointLoader 불러오기
from langchain_community.document_loaders import UnstructuredPowerPointLoader

#mode=elements를 통해 pptx의 요소별로 Document 객체로 가져오기
loader = UnstructuredPowerPointLoader(r"/content/drive/MyDrive/마소캠퍼스/Copilot-scenarios-for-Marketing.pptx", mode="elements")

#pptx 파일을 분할 로드하기
data = loader.load_and_split()

data[1]

In [ ]:
for i in data:
    if i.metadata['page_number'] == 2:
        print(i.metadata['category'])
        print(i.page_content)
        print("\n")

## **인터넷 정보 로드하기, WebBaseLoader**

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
#텍스트 추출할 URL 입력
loader = WebBaseLoader("https://www.espn.com/")
#ssl verification 에러 방지를 위한 코드
loader.requests_kwargs = {'verify':False}
data = loader.load()
data

**[Headline만 가져오기]**

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
#텍스트 추출할 URL 입력
loader = WebBaseLoader("https://www.espn.com/",
                        bs_kwargs=dict(
                            parse_only=bs4.SoupStrainer(
                                class_=("headlineStack top-headlines")
                                                        )
                                        )
                      )
#ssl verification 에러 방지를 위한 코드
loader.requests_kwargs = {'verify':False}
data = loader.load()
data

In [ ]:
loader = WebBaseLoader(["https://www.espn.com/", "https://google.com"])
docs = loader.load()
docs

## **특정 경로 내의 모든 파일 불러오기, DirectoryLoader**

In [ ]:
pip install unstructured[pdf]

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
#첫번째 매개변수로 경로 입력, glob에 해당 경로에서 불러들일 파일의 형식 지정
#*는 모든 문자를 표현하는 와일드카드로, .pdf로 끝나는 모든 파일을 의미함
loader = DirectoryLoader(r'/content/drive/MyDrive/마소캠퍼스', glob="*.pdf")
docs = loader.load()
[i.metadata['source'] for i in docs]

## **Text Splitters**

### **단순 글자수 기반 문서 분할, CharacterTextSplitter**

**[Chunk Overlap 개념 알아보기]**

In [ ]:
#Langchain Text Splitter 모듈 다운로드
!pip install -q langchain-text-splitters

In [ ]:
#PyPDFium2Loader로 PDF 문서 로드하기
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

#CharacterTextSplitter 모듈 로드
from langchain_text_splitters import CharacterTextSplitter

#구분자: 줄넘김, 청크 길이: 500, 청크 오버랩: 100, length_function: 글자수
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)
#텍스트 분할
texts = text_splitter.split_documents(pages)
print(texts[0])

In [ ]:
print(texts[1])

In [ ]:
loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

pages = loader.load()

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(pages)
print([len(i.page_content) for i in texts])

### **재귀적 문서 분할, RecursiveCharacterTextSplitter**

In [ ]:
from langchain_community.document_loaders import PyPDFium2Loader

loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")

pages = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter (
    separators=["\n\n", "\n", " ", ""],
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(pages)
print([len(i.page_content) for i in texts])

### **문맥 파악 통한 문서 분할, Semantic Chunker**

In [ ]:
pip install langchain_experimental

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
text_splitter = SemanticChunker(OpenAIEmbeddings(openai_api_key = "YOUR_API_KEY"))

texts = text_splitter.split_documents(pages)
print("-"*100)
print("[첫번째 청크]")
print(texts[0].page_content)
print("-"*100)
print("[두번째 청크]")
print(texts[1].page_content)

In [ ]:
print([len(i.page_content) for i in texts])

## **문장을 숫자로 바꾸자, Text Embedding**

### **OpenAI의 텍스트 임베딩 모델 활용하기**

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [ ]:
embeddings[0]

[-0.019139237701892853,
 -0.03814302384853363,
 -0.03093702718615532,
 -0.004656130913645029,
 -0.03535273298621178,
 -0.003945012576878071,
 0.013010076247155666,
 0.05103796720504761,
 -0.005804079119116068,
 -0.0371948666870594,
 -0.010754816234111786,
 -0.00218076235614717,
 0.02723921276628971,
 -0.002214625244960189,
 0.005895508453249931,
 0.03394404053688049,
 -0.016538577154278755,
 -0.01009787805378437,
 -0.03174973279237747,
 0.07655695080757141,
 0.05992355942726135,
 -0.018746430054306984,
 0.0030154797714203596,
 0.01899024099111557,
 0.03979552909731865,
 0.04578246548771858,
 0.020845921710133553,
 0.006549059879034758,
 0.013355476781725883,
 -0.0047475602477788925,
 0.029907599091529846,
 -0.022322338074445724,
 0.006941867992281914,
 -0.024029022082686424,
 -0.015007980167865753,
 -0.0035488184075802565,
 -0.007524307817220688,
 0.018123354762792587,
 -0.009671207517385483,
 -0.04250454902648926,
 0.012102554552257061,
 -0.018732884898781776,
 0.022308792918920517,
 

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#임베딩 모델 API 호출
embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

#PDF 문서 로드
loader = PyPDFium2Loader(r"/content/drive/MyDrive/마소캠퍼스/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

#PDF 문서를 여러 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

texts = text_splitter.split_documents(pages)

#OpenAI 임베딩 모델로 청크들을 임베딩 변환하기
embeddings = embeddings_model.embed_documents([i.page_content for i in texts])
len(embeddings), len(embeddings[0])

(53, 1536)

**[문장 유사도 계산해보기]**

In [ ]:
examples= embeddings_model.embed_documents(
     [
        "안녕하세요",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다.",
     ]
 )

#예시 질문과 답변 임베딩
embedded_query_q = embeddings_model.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = embeddings_model.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_a, examples [1]))
print(cos_sim(embedded_query_a, examples [3]))

0.955454149782215
0.9322697302217915
0.9105264737118787


### **오픈소스 임베딩 모델 활용하기**

**[jhgan/ko-sroberta-multitask 임베딩 모델 활용]**

In [ ]:
pip install sentence-transformers

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

#HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "jhgan/ko-sroberta-multitask"
ko_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)

examples = ko_embedding.embed_documents(
     [
        "안녕하세요",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다.",
     ]
 )

embedded_query_q = ko_embedding.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = ko_embedding.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, examples[1]))
print(cos_sim(embedded_query_q, examples[3]))

0.6070006291210956
0.2947341954203977
0.27578404901923914


**[BAAI/bge-small-en 임베딩 모델 활용 코드]**

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-small-en"
bge_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)

examples = bge_embedding.embed_documents(
     [
        "안녕하세요",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다.",
     ]
 )

embedded_query_q = bge_embedding.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = bge_embedding.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, examples[1]))
print(cos_sim(embedded_query_q, examples[3]))

0.955454149782215
0.9431682731233773
0.8853417113547791


## **문서 벡터 저장소, Vector Stores**

### **Langchain-Chroma 문서 저장 및 유사 문서 검색**

In [ ]:
pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 7.8 MB/s eta 0:00

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, openai_embedding)

In [ ]:
query = "대통령의 임기는?"
#유사 문서 검색
docs = db.similarity_search(query)
docs

[Document(metadata={'page': 10, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정\n한 국무위원의 순서로 그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책\n을 국민투표에 붙일 수 있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강\n화를 한다.'),
 Document(metadata={'page': 10, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='법제처                                                            11                                                       국가법령정보센터\n「대한민국헌법」 \n③탄핵소추의 의결을 받은 자는 탄핵심판이 있을 때까지 그 권한행사가 정지된다.\n④탄핵결정은 공직으로부터 파면함에 그친다. 그러나, 이에 의하여 민사상이나 형사상의 책임\n이 면제되지는 아니한다.\n \n                    제4장 정부\n                       제1절 대통령\n \n제66조 ①대통령은 국가의 원수이며, 외국에 대하여 국가를 대표한다.\n②대통령은 국가의 독립ㆍ영토의 보전ㆍ국가의 계속성과 헌법을 수호할 책무를 진다.\n③대통령은 조국의 평화적 통일을 위한 성실한 의무를 진다.\n④행정권은 대통령을 수반으로 하는 정부에 속한다.\n \n제67조 ①대통령은 국민의 보통ㆍ평등ㆍ직접ㆍ비밀선거에 의하여 선출한다.'

In [ ]:
#유사 문서 검색 및 유사도 출력
db.similarity_search_with_score(query)

[(Document(metadata={'page': 10, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.\n \n제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정\n한 국무위원의 순서로 그 권한을 대행한다.\n \n제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책\n을 국민투표에 붙일 수 있다.\n \n제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강\n화를 한다.'),
  1.0991065502166748),
 (Document(metadata={'page': 10, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='법제처                                                            11                                                       국가법령정보센터\n「대한민국헌법」 \n③탄핵소추의 의결을 받은 자는 탄핵심판이 있을 때까지 그 권한행사가 정지된다.\n④탄핵결정은 공직으로부터 파면함에 그친다. 그러나, 이에 의하여 민사상이나 형사상의 책임\n이 면제되지는 아니한다.\n \n                    제4장 정부\n                       제1절 대통령\n \n제66조 ①대통령은 국가의 원수이며, 외국에 대하여 국가를 대표한다.\n②대통령은 국가의 독립ㆍ영토의 보전ㆍ국가의 계속성과 헌법을 수호할 책무를 진다.\n③대통령은 조국의 평화적 통일을 위한 성실한 의무를 진다.\n④행정권은 대통령을 수반으로 하는 정부에 속한다.\n \n제67조 ①대통령은 국민의 

**[벡터DB를 로컬 디스크에 저장하고 로드하기]**

In [ ]:
Chroma().delete_collection()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)


#HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "jhgan/ko-sroberta-multitask"
ko_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)


#save to disk
db2 = Chroma.from_documents(docs, ko_embedding, persist_directory="./chroma_db")

In [ ]:
# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=ko_embedding)

query = "대통령의 임기는?"
result = db3.similarity_search(query)
print(result[0].page_content)

제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정
한 국무위원의 순서로 그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책
을 국민투표에 붙일 수 있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강
화를 한다.


In [ ]:
# # load from disk
# db3 = Chroma(persist_directory="./chroma_db", embedding_function=ko_embedding)

# query = "대통령의 임기는?"
# result = db3.similarity_search(query)
# print(result[0].page_content)

## **RAG의 핵심, 문서 검색기 Retriever**

### **Retriever의 기본형, 벡터DB 기반 Retriever**

**Chroma 벡터 DB 기반 기본 유사 문서 검색**

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

#Chroma를 Retriever로 활용
retriever = db.as_retriever()
retriever.invoke("국회의원의 의무")

[Document(metadata={'page': 7, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기중 국회의 동의없이 체포 또는 구금되\n지 아니한다.\n②국회의원이 회기전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면\n회기중 석방된다.\n \n제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회외에서 책임을 지지 아니한\n다.\n \n제46조 ①국회의원은 청렴의 의무가 있다.\n②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.\n③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하\n여 재산상의 권리ㆍ이익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대\n통령 또는 국회재적의원 4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.'),
 Document(metadata={'page': 9, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='한 동의권을 가진다.\n \n제61조 ①국회는 국정을 감사하거나 특정한 국정사안에 대하여 조사할 수 있으며, 이에 필요한\n서류의 제출 또는 증인의 출석과 증언이나 의견의 진술을 요구할 수 있다.\n②국정감사 및 조사에 관한 절차 기타 필요한 사항은 법률로 정한다.\n \n제62조 ①국무총리ㆍ국무위원 또는 정부위원은 국회나 그 위원회에 출석하여 국정처리상황을\n보고하거나 의견을 진술하고 질문에 응답할 수 있다.\n②국회나 그 위원회의 요

**(1) 검색 결과 수 및 조정**

In [ ]:
#유사 청크 1개만 반환
retriever = db.as_retriever(search_kwargs={"k": 10})
retriever.invoke("국회의원의 의무")

[Document(metadata={'page': 7, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기중 국회의 동의없이 체포 또는 구금되\n지 아니한다.\n②국회의원이 회기전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면\n회기중 석방된다.\n \n제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회외에서 책임을 지지 아니한\n다.\n \n제46조 ①국회의원은 청렴의 의무가 있다.\n②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.\n③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하\n여 재산상의 권리ㆍ이익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대\n통령 또는 국회재적의원 4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.'),
 Document(metadata={'page': 9, 'source': '/content/drive/MyDrive/마소캠퍼스/대한민국 헌법.pdf'}, page_content='한 동의권을 가진다.\n \n제61조 ①국회는 국정을 감사하거나 특정한 국정사안에 대하여 조사할 수 있으며, 이에 필요한\n서류의 제출 또는 증인의 출석과 증언이나 의견의 진술을 요구할 수 있다.\n②국정감사 및 조사에 관한 절차 기타 필요한 사항은 법률로 정한다.\n \n제62조 ①국무총리ㆍ국무위원 또는 정부위원은 국회나 그 위원회에 출석하여 국정처리상황을\n보고하거나 의견을 진술하고 질문에 응답할 수 있다.\n②국회나 그 위원회의 요

**(2) 검색 방식 변경 - MMR**

MMR은 쿼리에 대한 (1) 각 문서의 유사성 점수와 (2) 이미 선택된 문서들과의 다양성점수를 조합하여, 각 문서의 최종 점수를 계산합니다.

$\text{MMR} = \lambda \cdot \text{Sim}(d, Q) - (1 - \lambda) \cdot \max_{d' \in D'} \text{Sim}(d, d')$

즉 유사 문서 후보군 중 특정 문서가 후보군 내 문서들 간의 유사성은 낮고, 쿼리와의 유사성이 높은 경우 점수가 높습니다.

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
#헌법 PDF 파일 로드
loader = PyPDFLoader(r"C:\Users\gram\Downloads\대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

In [ ]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs = {"lambda_mult": 0, "fetch_k":10, "k":3}
)
result = retriever.invoke("국회의원의 의무")
for idx, value in enumerate(result):
  print(f"{idx+1}번째 유사 문서:")
  print(value.page_content[:100])
  print("\n\n")

1번째 유사 문서:
제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.
 
제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기중 국회의 동의없이 체포 또는 구금되
지 아니한다.
②국회의원이 회



2번째 유사 문서:
⑦각급 선거관리위원회의 조직ㆍ직무범위 기타 필요한 사항은 법률로 정한다.
 
제115조 ①각급 선거관리위원회는 선거인명부의 작성등 선거사무와 국민투표사무에 관하여 관
계 행정기관에



3번째 유사 문서:
법제처                                                            20                                   





**일반 유사도 검색 방식**

In [ ]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(search_kwargs = {"k":3})
result = retriever.invoke("국회의원의 의무")
for idx, value in enumerate(result):
  print(f"{idx+1}번째 유사 문서:")
  print(value.page_content[:100])
  print("\n\n")

1번째 유사 문서:
제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.
 
제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기중 국회의 동의없이 체포 또는 구금되
지 아니한다.
②국회의원이 회



2번째 유사 문서:
한 동의권을 가진다.
 
제61조 ①국회는 국정을 감사하거나 특정한 국정사안에 대하여 조사할 수 있으며, 이에 필요한
서류의 제출 또는 증인의 출석과 증언이나 의견의 진술을 요구할



3번째 유사 문서:
제64조 ①국회는 법률에 저촉되지 아니하는 범위안에서 의사와 내부규율에 관한 규칙을 제정할
수 있다.
②국회는 의원의 자격을 심사하며, 의원을 징계할 수 있다.
③의원을 제명하려면



